# Install

In [ ]:
!pip3 install pshmodule

In [ ]:
!pip3 install pickle5

In [ ]:
!pip3 install pandas==1.5.0

In [ ]:
!pip3 install swifter

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Load

In [1]:
import sys
sys.path.append('/content/drive/MyDrive/Mbti_LexicalTendency/src/')
print(sys.path)

['/content', '/env/python', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/usr/local/lib/python3.9/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.9/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/MyDrive/Mbti_LexicalTendency/src/']


In [2]:
from tqdm import tqdm
import config as cfg
import pandas as pd
import swifter
from pshmodule.utils import filemanager as fm

In [3]:
df_nf = pd.read_excel('/content/drive/MyDrive/Mbti_LexicalTendency/data/mbti_pattern.xlsx', sheet_name='NF_경청')
df_st = pd.read_excel('/content/drive/MyDrive/Mbti_LexicalTendency/data/mbti_pattern.xlsx', sheet_name='ST_경청')
df_sf = pd.read_excel('/content/drive/MyDrive/Mbti_LexicalTendency/data/mbti_pattern.xlsx', sheet_name='SF_경청')
df_nt = pd.read_excel('/content/drive/MyDrive/Mbti_LexicalTendency/data/mbti_pattern.xlsx', sheet_name='NT_경청')


df_nf.dropna(inplace=True)
df_st.dropna(inplace=True)
df_sf.dropna(inplace=True)
df_nt.dropna(inplace=True)

In [4]:
print(df_nf.shape)
print(df_st.shape)
print(df_sf.shape)
print(df_nt.shape)

(461, 3)
(1219, 3)
(1180, 3)
(1408, 3)


# Check Algorithm

In [5]:
df_sf.head()

,유형번호,패턴조합,출현빈도
0,1,경청 표시-경청 표시-경청 표시-경청 표시-경청 표시-경청 표시-경청 표시-경청 표...,6
1,2,경청 표시-경청 표시-경청 표시-경청 표시-경청 표시-경청 표시-경청 표시-경청 표시,5
2,3,경청 표시-경청 표시-경청 표시-경청 표시-경청 표시-경청 표시-경청 표시-경청 표...,5
3,4,경청 표시-경청 표시-경청 표시-경청 표시-경청 표시-경청 표시-경청 표시-경청 표...,4
4,5,경청 표시-경청 표시-경청 표시-경청 표시-경청 표시-경청 표시-경청 표시-경청 표...,4


공유드린 곳에 들어가면 아까 보셨던 "패턴조합"이 있고요<br>
제가 찾고 싶은 것은  이 안에서<br>
"뽑혀 나온 패턴 들 중에서 '경청표시-느낌표시-부연-동의' 인 것은?"<br><br>

(조건)<br>
네 개의 패턴표지의 순서는 지켜져야 함.<br>
표지의 순서가 유지된다면, 표지 전후에 다른 표지가 끼어들거나 일부표지가 반복되어 나타나도 상관없음.<br><br>

(정답/오답 예시)<br>
예를 들어 "경청-느낌-느낌-경청-부연-동의" <- 이것은 정답이고 (순서 O, 표지 포함O)<br>
"동의-부연-경청-느낌-동의-부연-동의" <- 이것은 정답 (순서 O, 표지 포함O)<br>
"느낌-느낌-경청-부연" <- 이것은 오답 (순서X , 표지 포함X)<br>
"경청-부연-동의" <- 이것은 오답(순서 O, 표지 포함X)<br>

패턴 찾기

In [6]:
pattern = {'경청': 0, '느낌': 1, '부연': 2, '동의': 3}

In [7]:
# find sequence
def find_subsequence(subsequence, mainsequence):
  sub_idx = 0
  for item in mainsequence:
      if item == subsequence[sub_idx]:
          sub_idx += 1
          if sub_idx == len(subsequence):
              return True
  return False

In [8]:
# pattern check
class pattern_check:
  def __init__(self, df):
    self.df = df
    self.result = []

  def pattern(self):
    for i in self.df.iterrows():
      check = []

      # - split 후 중복 제거  
      temp = ['']
      for p in i[1]['패턴조합'].split('-'):
        if temp[-1] != p:
          temp.append(p)
      temp.remove('')

      # 번호 매기기
      for t in temp:
        for s in t.split():
          if s in pattern.keys():
            check.append(pattern[s])


      # 순서대로 있는지 체크
      check_temp = find_subsequence([0, 1, 2, 3], check)
      self.result.append(str(check_temp))
      
    return self.result

In [13]:
p_nf = pattern_check(df_nf)
p_st = pattern_check(df_st)
p_sf = pattern_check(df_sf)
p_nt = pattern_check(df_nt)

In [14]:
result_nf = p_nf.pattern()
result_st = p_st.pattern()
result_sf = p_sf.pattern()
result_nt = p_nt.pattern()

In [17]:
print(len(result_nf))
print(len(result_st))
print(len(result_sf))
print(len(result_nt))

461
1219
1180
1408


In [18]:
df_nf['result'] = result_nf
df_st['result'] = result_st
df_sf['result'] = result_sf
df_nt['result'] = result_nt

In [19]:
print(df_nf[df_nf.result=='True'].shape)
print(df_st[df_st.result=='True'].shape)
print(df_sf[df_sf.result=='True'].shape)
print(df_nt[df_nt.result=='True'].shape)

(27, 4)
(49, 4)
(26, 4)
(32, 4)


# Save

In [24]:
fm.save('/content/drive/MyDrive/Mbti_LexicalTendency/data/reports/pattern_nf.xlsx', df_nf)
fm.save('/content/drive/MyDrive/Mbti_LexicalTendency/data/reports/pattern_st.xlsx', df_st)
fm.save('/content/drive/MyDrive/Mbti_LexicalTendency/data/reports/pattern_sf.xlsx', df_sf)
fm.save('/content/drive/MyDrive/Mbti_LexicalTendency/data/reports/pattern_nt.xlsx', df_nt)

Saved 461 records
Saved 1219 records
Saved 1180 records
Saved 1408 records


In [23]:
# #1. 파일 생성
# writer = pd.ExcelWriter('/content/drive/MyDrive/Mbti_LexicalTendency/data/reports/pattern_mbti.xlsx', engine='openpyxl')
 
# #2. 생성 파일에 시트명 지정 후 dataframe에 저장한 결과값 넣기
# df_nf.to_excel(writer, sheet_name='NF_경청패턴')
# df_st.to_excel(writer, sheet_name='ST_경청패턴')
# df_sf.to_excel(writer, sheet_name='SF_경청패턴')
# df_nt.to_excel(writer, sheet_name='NT_경청패턴')
 
# #3. 작성 완료 후 파일 저장
# writer.save()